In [ ]:
import requests
import json
import os

# URL to send the POST request to
external_url = "https://ml-b74f8940-b97.go01-dem.ylcu-atmi.cloudera.site/"
internal_url = "https://127.0.0.1:" + str(os.environ['CDSW_APP_PORT']) + "/"

# Choose which URL to use
url = internal_url

# Headers for the POST request
headers = {
    "Content-Type": "application/json"
}

# JSON Body for the POST request
payload = {
    "inputs": "What is Cloudera Data Science Workbench?",
    "parameters": {
        "temperature": 1,
        "max_tokens": 10
    }
}

# Convert Python dictionary to JSON
payload_json = json.dumps(payload)

# Make the POST request
response = requests.post(url, headers=headers, data=payload_json)

# Check if the request was successful
if response.status_code == 200:
    print(f"Success! Received response: {response.json()}")
else:
    print(f"Failed to make request. Status code: {response.status_code}, Response: {response.text}")


https://127.0.0.1:8090/
